In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,f1_score
from sentence_transformers import SentenceTransformer
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import pipeline
import torch
import pandas as pd
from tqdm import tqdm

In [ ]:
X_train = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/X_train.csv")
y_train = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/y_train.csv")
X_test = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/X_test.csv")
y_test = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/y_test.csv")

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = "tuner007/pegasus_paraphrase"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

def paraphrase_text(text, max_length=100):
    prompt = f"Rewrite the following differently: {text}"  # Prompt ekleyerek daha iyi sonuç alınabilir.

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_return_sequences=4,  # Çeşitli paraphrasing seçenekleri almak için
        do_sample=True,  # Sampling aktif edildi
        temperature=1.8,  # Çıktılarda çeşitlilik için sıcaklık artırıldı
        top_k=50,
        top_p=0.9,
        repetition_penalty=2.0,
        num_beams=5  # Beam search ile çeşitliliği artır
    )

    return [tokenizer.decode(output, skip_special_tokens=True).strip() for output in outputs]

# X_train veri setinde tüm metinler için paraphrasing işlemi
def generate_paraphrased_dataframe(X_train):
    data = []

    for text in tqdm(X_train, desc="Generating paraphrases"):
        paraphrased_texts = paraphrase_text(text)
        row = [text] + paraphrased_texts  # Orijinal + 4 paraphrased versiyon
        data.append(row)

    # DataFrame oluşturma
    df = pd.DataFrame(data, columns=['original', 'augmented_1', 'augmented_2', 'augmented_3', 'augmented_4'])
    return df

df_paraphrased = generate_paraphrased_dataframe(X_train.values.flatten())

In [ ]:
df_paraphrased.to_csv("/content/drive/MyDrive/Müş/LLM_Aug/pegasus_augmented.csv",index=False)

In [ ]:
import pandas as pd
df_paraphrased = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/pegasus_augmented.csv")

In [ ]:
df_paraphrased.iloc[1].values

In [ ]:
df_paraphrased

In [ ]:
df_augmented = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/pegasus_augmented.csv")

In [ ]:
df_2x = pd.concat([df_augmented[['original']], df_augmented[['augmented_1']].rename(columns={"augmented_1": "original"})], ignore_index=True)
y_expanded = pd.concat([y_train, y_train], ignore_index=True)
# Yeni sütunu df_2x'e ekleyelim
df_2x["y"] = y_expanded.values
df_2x

In [ ]:
df_3x = pd.concat([
    df_augmented[['original']],
    df_augmented[['augmented_1']].rename(columns={"augmented_1": "original"}),
    df_augmented[['augmented_2']].rename(columns={"augmented_2": "original"})
], ignore_index=True)

# y_train etiketlerini tekrarlayarak ekle
y_expanded = pd.concat([y_train, y_train, y_train], ignore_index=True)
df_3x["y"] = y_expanded.values
df_3x

In [ ]:
df_5x = pd.concat([
    df_augmented[['original']],
    df_augmented[['augmented_1']].rename(columns={"augmented_1": "original"}),
    df_augmented[['augmented_2']].rename(columns={"augmented_2": "original"}),
    df_augmented[['augmented_3']].rename(columns={"augmented_3": "original"}),
    df_augmented[['augmented_4']].rename(columns={"augmented_4": "original"})
], ignore_index=True)

# y_train değerlerini 5 kez tekrar ederek ekle
y_expanded = pd.concat([y_train] * 5, ignore_index=True)

# Yeni y sütununu ekle
df_5x["y"] = y_expanded.values
df_5x

In [ ]:
y_2x_train=df_2x["y"]
y_3x_train=df_3x["y"]
y_5x_train=df_5x["y"]

## Embeddings

In [ ]:
X_test_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2").encode(X_test["Text"].tolist(),batch_size=32,show_progress_bar=True)

In [ ]:
X_original_train_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2").encode(X_train["Text"].tolist(),batch_size=32,show_progress_bar=True)


In [ ]:
X_2x_train_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2").encode(df_2x["original"].tolist(),batch_size=32,show_progress_bar=True)

In [ ]:
X_3x_train_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2").encode(df_3x["original"].tolist(),batch_size=32,show_progress_bar=True)

In [ ]:
X_5x_train_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2").encode(df_5x["original"].tolist(),batch_size=32,show_progress_bar=True)

## Model Eğitimi

In [ ]:
#Original Data
# XGBoost modelini oluştur
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Çok sınıflı sınıflandırma için
    num_class=len(np.unique(y_train)), # Sınıf sayısını belirle
    eval_metric='mlogloss',      # Çok sınıflı log loss metriği
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    use_label_encoder=False
)

# Modeli eğit
xgb_model.fit(X_original_train_embeddings, y_train)
# Test seti ile tahmin yap
y_pred = xgb_model.predict(X_test_embeddings)

# Model doğruluğunu hesapla
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu: {accuracy:.2f}")

f1score = f1_score(y_test, y_pred,average="macro")
print(f"F1-Score: {f1score:.2f}")

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0","1","2","3","4"], yticklabels=["0","1","2","3","4"])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Sınıflandırma raporunu yazdır
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred))

In [ ]:
#2x Data
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Çok sınıflı sınıflandırma için
    num_class=len(np.unique(y_train)), # Sınıf sayısını belirle
    eval_metric='mlogloss',      # Çok sınıflı log loss metriği
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    use_label_encoder=False
)

# Modeli eğit
xgb_model.fit(X_2x_train_embeddings, y_2x_train)
# Test seti ile tahmin yap
y_pred = xgb_model.predict(X_test_embeddings)

# Model doğruluğunu hesapla
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu: {accuracy:.2f}")

f1score = f1_score(y_test, y_pred,average="macro")
print(f"F1-Score: {f1score:.2f}")

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0","1","2","3","4"], yticklabels=["0","1","2","3","4"])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Sınıflandırma raporunu yazdır
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred))

In [ ]:
#3x Data
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Çok sınıflı sınıflandırma için
    num_class=len(np.unique(y_train)), # Sınıf sayısını belirle
    eval_metric='mlogloss',      # Çok sınıflı log loss metriği
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    use_label_encoder=False
)

# Modeli eğit
xgb_model.fit(X_3x_train_embeddings, y_3x_train)
# Test seti ile tahmin yap
y_pred = xgb_model.predict(X_test_embeddings)

# Model doğruluğunu hesapla
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu: {accuracy:.2f}")

f1score = f1_score(y_test, y_pred,average="macro")
print(f"F1-Score: {f1score:.2f}")

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0","1","2","3","4"], yticklabels=["0","1","2","3","4"])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Sınıflandırma raporunu yazdır
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred))

In [ ]:
#5x Data
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Çok sınıflı sınıflandırma için
    num_class=len(np.unique(y_train)), # Sınıf sayısını belirle
    eval_metric='mlogloss',      # Çok sınıflı log loss metriği
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    use_label_encoder=False
)

# Modeli eğit
xgb_model.fit(X_5x_train_embeddings, y_5x_train)
# Test seti ile tahmin yap
y_pred = xgb_model.predict(X_test_embeddings)

# Model doğruluğunu hesapla
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu: {accuracy:.2f}")

f1score = f1_score(y_test, y_pred,average="macro")
print(f"F1-Score: {f1score:.2f}")

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0","1","2","3","4"], yticklabels=["0","1","2","3","4"])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Sınıflandırma raporunu yazdır
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred))